In [4]:
import numpy as np
import os
from ase.io import read, write
from matscipy.neighbours import neighbour_list
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import entropy, circvar

def dihedral(p1, p2, p3, p4):
    b0 = -(p2 - p1)
    b1 = (p3 - p2)
    b2 = (p4 - p3)
    b1 /= np.linalg.norm(b1)
    v = b0 - np.dot(b0, b1) * b1
    w = b2 - np.dot(b2, b1) * b1
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

def renyi_entropy(prob, alpha=2):
    prob = np.array(prob)
    prob = prob[prob > 0] 
    return 1 / (1 - alpha) * np.log2(np.sum(prob ** alpha))

def is_crossing_mic(p_ref, p_target, cell):
    delta = p_target - p_ref
    inv_cell = np.linalg.inv(cell.T)
    frac = np.dot(delta, inv_cell)
    return np.any(np.abs(frac) > 0.5)

def plot_polar_dihedral_distribution(angles, color='#d3d3d3', num_bins=60): 
    angles = (np.array(angles) + 360) % 360
    bin_width = 360 / num_bins
    half = bin_width / 2
    edges_deg = np.linspace(-half, 360 - half, num_bins + 1)
    hist, _ = np.histogram(angles, bins=edges_deg, density=True)
    centers_rad = np.radians(edges_deg[:-1] + half)

    fig = plt.figure(figsize=(2.2, 2.2))
    ax = fig.add_subplot(111, polar=True)
    ax.set_theta_zero_location("S")
    ax.set_theta_direction(1)
    ax.set_thetamin(-180)
    ax.set_thetamax(180)
    ax.yaxis.grid(True)
    ax.set_yticks(np.linspace(0, 0.007, 4))
    ax.yaxis.grid(True, linestyle='--', linewidth=0.5, color='gray', zorder=0)
    ax.xaxis.grid(True, linestyle='--', linewidth=0.5, color='gray', zorder=0)
    ax.bar(centers_rad, hist, width=np.radians(bin_width), align='center',
           alpha=0.8, color=color, zorder=5)
    ax.set_thetagrids(
        angles=[-135, -90, -45, 0, 45, 90, 135, 180],
        labels=["-135°", "-90°", "-45°", "0°", "45°", "90°", "135°", "±180°"]
    )
    ax.set_yticklabels([])
    ax.set_rlim(0, 0.007)
    plt.show()
    plt.close()

def calc_rotation_angle_distribution(dump_file, cutoff, color):
    atoms = read(dump_file, index=-1)
    cell = atoms.get_cell()
    i, j = neighbour_list('ij', atoms, cutoff=cutoff)
    nbrs = defaultdict(list)
    for a, b in zip(i, j):
        nbrs[a].append(b)

    seen_configs = set()
    angle_data = []

    for a in nbrs:
        if len(nbrs[a]) != 3:
            continue
        for d in nbrs[a]:
            if d <= a or len(nbrs[d]) != 3 or a not in nbrs[d]:
                continue
            others_a = tuple(sorted(x for x in nbrs[a] if x != d))
            others_d = tuple(sorted(x for x in nbrs[d] if x != a))

            config_id = (a, d, others_a, others_d)
            if config_id in seen_configs:
                continue
            seen_configs.add(config_id)

            if len(others_a) < 2 or len(others_d) < 2:
                continue
            b, c = others_a[:2]
            e, f = others_d[:2]
            
            p2 = atoms.positions[a]
            vec_ab = atoms.get_distance(a, b, vector=True, mic=True)
            vec_ac = atoms.get_distance(a, c, vector=True, mic=True)
            ag = vec_ab + vec_ac
            p1 = p2 + ag
            vec_ad = atoms.get_distance(a, d, vector=True, mic=True)
            p3 = p2 + vec_ad
            vec_de = atoms.get_distance(d, e, vector=True, mic=True)
            vec_df = atoms.get_distance(d, f, vector=True, mic=True)
            dh = vec_de + vec_df
            p4 = p3 + dh
            angle = dihedral(p1, p2, p3, p4)
            angle_data.append((a, b, c, d, e, f, angle))

    angles = [x[-1] for x in angle_data]
    plot_polar_dihedral_distribution(angles, color)

    angles_mod = (np.array(angles) + 360) % 360
    num_bins = 60
    hist, _ = np.histogram(angles_mod, bins=num_bins, range=(0, 360), density=True)
    hist_safe = hist + 1e-12

    H = entropy(hist_safe, base=2)

    print(f"Shannon entropy: {H:.3f} bits")

In [ ]:
cutoffs  = [2.9, 2.4]

files = ['a-As', 'a-P']

colors = ['red', 'blue']

for file, cutoff, color in zip(files, cutoffs, colors):
    dump = f'{file}.extxyz'
    print("Processing", dump)
    calc_rotation_angle_distribution(dump, cutoff, color)